### 示例13-1

In [ ]:
from pyproj import CRS
from pyproj.enums import WktVersion
from osgeo import gdal,gdalconst,ogr

def Pixel2world(geotransform, line, column):
    originX = geotransform[0]
    originY = geotransform[3]
    pixelWidth = geotransform[1]
    pixelHeight = geotransform[5]
    x = (column-1)*pixelWidth + originX
    y = (line-1)*pixelHeight + originY
    return(x,y)

def sub_image(in_raster,out_raster,bound,bound_coor_system):
    """
    提取栅格数据子区。
    in_raster：输入栅格数据
    out_raster：输出栅格数据
    bound：子区范围（左下角和右上角坐标）
    bound_coor_system：子区范围坐标系统
    """
    from pyproj import Transformer
    from osgeo import gdal

    #空间坐标转行列坐标
    def world2Pixel(geotransform, x, y):
        originX = geotransform[0]
        originY = geotransform[3]
        pixelWidth = geotransform[1]
        pixelHeight = geotransform[5]
        line = int((y-originY)/pixelHeight)+1
        column = int((x-originX)/pixelWidth)+1
        return (line,column)
    
    src_ds = gdal.Open(in_raster)
    src_driver = src_ds.GetDriver()
    src_bands = src_ds.RasterCount
    src_dataType = src_ds.GetRasterBand(1).DataType
    src_geotransform = src_ds.GetGeoTransform()
    src_projection = src_ds.GetProjection()
    
    #子区左下角和右上角经纬度坐标转输入栅格数据的坐标系统，然后返回行列坐标
    minx,miny,maxx,maxy = bound
    LL = (minx,miny)
    UR = (maxx,maxy)
    transformer = Transformer.from_crs(bound_coor_system,src_projection,always_xy=True)
    LL_x,LL_y = transformer.transform(LL[0],LL[1])
    UR_x,UR_y = transformer.transform(UR[0],UR[1])
    LL_line,LL_column = world2Pixel(src_geotransform, LL_x, LL_y)
    UR_line,UR_column = world2Pixel(src_geotransform, UR_x, UR_y)
    
    #读子区数据
    array = src_ds.ReadAsArray(xoff=LL_column,yoff=UR_line,
                           xsize=UR_column-LL_column,
                           ysize=LL_line-UR_line)
    
    #如输入栅格数据是单波段数据，把二维数组设置成三维数组
    if src_bands == 1:
        array.shape = (1,LL_line-UR_line,UR_column-LL_column)

    #创建输出栅格数据，其中driver、bands、eType、Projection同输入栅格数据
    dst_driver = src_driver
    dst_ds = dst_driver.Create(out_raster,
                          UR_column-LL_column,LL_line-UR_line,
                          src_bands,src_dataType)
    dst_ds.SetProjection(src_projection)
    
    #设置新数据的坐标转换参数（修改左上角栅格的x和y坐标）
    new_gt = list(src_geotransform)
    new_gt[0] = LL_x
    new_gt[3] = UR_y
    dst_ds.SetGeoTransform(new_gt)
    
    #array写入到栅格数据中
    for i in range(src_bands):
        band = dst_ds.GetRasterBand(i+1)
        band.WriteArray(array[i])
    dst_ds.FlushCache()
    dst_ds = None

def bounds(in_raster,out_coor_system):
    """
    计算栅格数据坐标转换后的矩形范围（返回最小最大x和y坐标）
    in_raster：输入栅格数据
    out_coor_system：输出栅格数据坐标系统
    """
    from pyproj import Transformer
    from osgeo import gdal
    x_ts = [];y_ts = []    #边缘栅格坐标转换后的x和y坐标列表

    #获取输入栅格数据信息
    ds = gdal.Open(in_raster)
    lines = ds.RasterYSize 
    cols = ds.RasterXSize
    geotransform = ds.GetGeoTransform()
    projection = ds.GetProjection()
    #对每个边缘栅格的x、y坐标进行转换
    transformer = Transformer.from_crs(projection,out_coor_system,always_xy=True)
    for line in range(lines):
        #如果是第一行或最后一行，则计算每一列栅格
        if line == 0 or line == lines-1:
            for col in range(cols):
                x,y = Pixel2world(geotransform,line+1,col+1)
                x_t,y_t = transformer.transform(x,y)
                x_ts.append(x_t)
                y_ts.append(y_t)
        #如果不是第一行或最后一行，只计算第一列和最后一列栅格
        else:
            x,y = Pixel2world(geotransform, line+1,1)
            x_t,y_t = transformer.transform(x,y)
            x_ts.append(x_t)
            y_ts.append(y_t)
            x,y = Pixel2world(geotransform, line+1,cols)
            x_t,y_t = transformer.transform(x,y)
            x_ts.append(x_t)
            y_ts.append(y_t)
    #返回最小、最大x、y坐标
    x_t_min = round(min(x_ts),2)
    y_t_min = round(min(y_ts),2)
    x_t_max = round(max(x_ts),2)
    y_t_max = round(max(y_ts),2)    
    return (x_t_min,y_t_min,x_t_max,y_t_max)

def projectRaster(in_raster,out_raster,out_coor_system,x_size=None,y_size=None):
    """
    栅格数据坐标转换
    in_raster：输入栅格数据
    out_raster：输出栅格数据
    out_coor_system：输出栅格数据的坐标系统
    x_size，y_size：输出栅格x和y方向的大小，如未定义则按行列数和范围计算
    """
    from pyproj import Transformer
    from shapely.geometry import Polygon
    from shapely.ops import transform
    from osgeo import gdal
    from osgeo import gdal,gdalconst
    #获取输入栅格数据信息
    src_ds = gdal.Open(in_raster)
    src_driver = src_ds.GetDriver()
    src_lines = src_ds.RasterYSize 
    src_cols = src_ds.RasterXSize
    src_bands = src_ds.RasterCount
    src_dataType = src_ds.GetRasterBand(1).DataType 
    src_geotransform = src_ds.GetGeoTransform()
    src_projection = src_ds.GetProjection()
    
    #调用bounds()函数
    minx,miny,maxx,maxy = bounds(in_raster,out_coor_system)
   
    #计算输出栅格数据的行列数
    if y_size != None:
        dst_lines = int((maxy - miny)/y_size)
    else:
        dst_lines = src_lines
        y_size = (maxy - miny)/src_lines
    
    if x_size != None:
        dst_cols = int((maxx - minx)/x_size)
    else:
        dst_cols = src_cols
        x_size = (maxx - minx)/src_cols
    
    #创建输出栅格数据，其中driver、bands、eType同输入栅格数据
    dst_driver = src_driver
    dst_ds = dst_driver.Create(out_raster,
                          xsize=dst_cols,ysize=dst_lines,
                          bands = src_bands,
                          eType = src_dataType)
    dst_GeoTransform = (minx,x_size,0,maxy,0,-y_size)
    dst_ds.SetGeoTransform(dst_GeoTransform)
    dst_ds.SetProjection(out_coor_system)
    
    #对输出栅格数据进行重采样，并写到文件中
    gdal.Warp(dst_ds,src_ds)
    dst_ds.FlushCache()
    dst_ds = None
    
def mask_image(in_raster,out_raster,in_vector):
    """
    对栅格数据进行掩膜处理。本函数没有考虑坐标转换，
    要求输入的栅格数据和矢量数据的坐标系统一致。
    in_raster：输入栅格数据
    out_raster：掩膜处理后的栅格数据
    in_vector：用于掩膜的矢量数据，shapefile数据
    """
    from osgeo import gdal,gdalconst,ogr
   
    #使用ogr读矢量数据
    driver = ogr.GetDriverByName('ESRI Shapefile')
    vector_ds = driver.Open(in_vector,0)
    layer = vector_ds.GetLayer(0)

    #获取输入栅格数据参数
    in_ds = gdal.Open(in_raster)
    in_driver = in_ds.GetDriver()
    rows = in_ds.RasterYSize
    cols = in_ds.RasterXSize
    bands = in_ds.RasterCount
    geotransform = in_ds.GetGeoTransform()
    projection = in_ds.GetProjection()
    
    #矢量数据转栅格数据
    mask_raster = r"c:\data\tmp\tmp_mask.img"
    mask_ds = in_driver.Create(mask_raster,cols,rows,1,gdalconst.GDT_Byte)
    mask_ds.SetGeoTransform(geotransform)
    mask_ds.SetProjection(projection)
    gdal.RasterizeLayer(mask_ds,[1],layer)
    mask_array = mask_ds.ReadAsArray()
   
    #复制输入数据，把矢量数据范围外的栅格赋值为0，并设为空值
    out_ds = in_driver.CreateCopy(out_raster,in_ds)
    for i in range(bands):
        band = out_ds.GetRasterBand(i+1)
        out_array = band.ReadAsArray()
        out_array[mask_array == 0] = 0
        band.WriteArray(out_array)
        band.SetNoDataValue(0)
    out_ds.FlushCache()
    out_ds = None

#提取子区
in_raster = r"c:\data\VIIRS\SVDNB_npp_20150101-20151231_75N060E_vcm-orm-ntl_v10_c201701311200.avg_rade9.tif"
sub_raster = r"c:\data\VIIRS\sub_image.tif"
in_vector_wgs =r'c:\data\VIIRS\csj_WGS.shp'
driver = ogr.GetDriverByName('ESRI Shapefile')
vector_ds = driver.Open(in_vector_wgs, 0)
layer = vector_ds.GetLayer(0)
minx,maxx,miny,maxy = layer.GetExtent()
bound = (minx,miny,maxx,maxy)
crs = 4326
sub_image(in_raster,sub_raster,bound,crs)

#坐标系统转换
aea = "+proj=aea +lat_1=25 +lat_2=45 +lat_0=0 +lon_0=105"
crs = CRS.from_proj4(aea)
aea_wkt = crs.to_wkt(version=WktVersion.WKT1_GDAL,pretty=True)
projected_raster = r"c:\data\VIIRS\projected_image.tif"
projectRaster(sub_raster,projected_raster,aea_wkt,x_size=500,y_size=500)

#掩膜处理
in_vector_aea = r'c:\data\VIIRS\csj_aea.shp'
masked_raster = r"c:\data\VIIRS\masked_image.tif"
mask_image(projected_raster,masked_raster,in_vector_aea)

### 示例13-2

In [ ]:
def value_count(in_raster):
    from osgeo import gdal
    import numpy as np
    ds = gdal.Open(in_raster)
    band1 = ds.GetRasterBand(1)
    array = band1.ReadAsArray()
    
#转换成一维数据，数据类型转换成整型
    flatten_array = array.flatten().astype(np.uint16)
    bincount = np.bincount(flatten_array)
    for i in range(len(bincount)):
        if bincount[i] == 0:
            continue
        else:
            print(i," ",bincount[i])

#以下代码是统计masked_image.tif数据中不同栅格值出现的频率：
in_raster = "c:\data\VIIRS\masked_image.tif"
value_count(in_raster)

### 示例13-3

In [ ]:
def outlier(in_data,out_data,threshold):
    """
    异常栅格检测函数
    in_data：输入栅格数据
    out_data：输出csv文件
    threshold：阈值
    """
    from osgeo import gdal
    import numpy as np
    from pyproj import Transformer
    
    def Pixel2world(geotransform, line, column):
        originX = geotransform[0]
        originY = geotransform[3]
        pixelWidth = geotransform[1]
        pixelHeight = geotransform[5]
        x = column*pixelWidth + originX - pixelWidth/2
        y = line*pixelHeight + originY - pixelHeight/2
        return(x,y)

    ds = gdal.Open(in_data)
    geotransform = ds.GetGeoTransform()
    projection = ds.GetProjection()
    transformer = Transformer.from_crs(projection,4326,always_xy=True)
    array = ds.ReadAsArray()
    matrix = np.matrix(array)
    #返回符合条件的栅格索引列表
    pixels = np.argwhere(matrix > threshold)

    points = []
    i = 0
    for pixel in pixels:
        i= i + 1
        line = pixel[0]     #行号
        column = pixel[1]   #列号
        value = int(array[pixel[0],pixel[1]])
        x,y = Pixel2world(geotransform, line, column)
        lng,lat = transformer.transform(x,y)
        points.append([value,line,column,lng,lat])

    s_points = sorted(points,reverse=True) #按value值从大到小排序
    seq = ""
    i = 0
    for s_point in s_points:
        i = i + 1
        seq = seq + str(i) + "," + str(s_point[0]) + "," +\
        str(s_point[1]) + "," + str(s_point[2]) +"," +\
        str(s_point[3]) + "," + str(s_point[4]) + "\n"        
    output_file = open(out_data,"w")
    output_file.writelines(seq)
    output_file.close()

def outlier_map(csv_data):
    """
    异常点地图可视化
    """
    import folium
    from IPython.display import display
    m = folium.Map(width = 600,height = 400,
                  zoom_start=11)
    file = open(csv_data,"r")
    lngs = [];lats = []   #用于确定地图显示范围
    for row in file:
        order,value,line,column,lng,lat = row.split(",")
        lngs.append(float(lng));lats.append(float(lat))
        html = f"""序号：{order}<br>
               栅格值：{value}"""
        popup = folium.Popup(html,max_width=200)
        folium.Marker(location = [float(lat),float(lng)],
                    popup=popup).add_to(m)
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
    folium.TileLayer(tiles=tiles,
                 name="遥感影像",
                 attr="arcgisonline全球影像",
                 overlay=False).add_to(m)

    folium.LayerControl().add_to(m)
    m.fit_bounds([(min(lats),min(lngs)),(max(lats),max(lngs))])
    display(m)

#以下代码是对示例13-1得到的夜间灯光数据进行异常栅格检测（设置阈值为180）：
in_raster = r"c:\data\VIIRS\masked_image.tif"
csv_data = r"c:\data\VIIRS\outlier.csv"
threshold = 180
outlier(in_raster,csv_data,threshold)
outlier_map(csv_data)

### 示例13-4

In [ ]:
from osgeo import gdal
import numpy as np
in_raster = "c:\data\VIIRS\masked_image.tif"
csv_data = r"c:\data\VIIRS\outlier.csv"
out_raster = "c:\data\VIIRS\processed_image.tif"
in_ds = gdal.Open(in_raster)
in_driver = in_ds.GetDriver()
out_ds = in_driver.CreateCopy(out_raster,in_ds)
band = out_ds.GetRasterBand(1)
array = band.ReadAsArray()
file = open(csv_data,"r")
#根据csv文件中的记录对每个异常栅格进行处理
for row in file:
    line = int(row.strip("\n").split(",")[2])
    col = int(row.strip("\n").split(",")[3]) 
    type_id = int(row.strip("\n").split(",")[6])
    if type_id == 1:
        array[line,col] = 0
    elif type_id == 2:
        window_size = 3    #滤波窗口大小3*3
        line_start = line - int((window_size-1)/2)
        line_end = line + int((window_size-1)/2) + 1
        col_start = col - int((window_size-1)/2)
        col_end = col + int((window_size-1)/2) + 1
        array[line,col] = np.median(array[line_start:line_end,
                                   col_start:col_end])
band.WriteArray(array)
file.close()
out_ds.FlushCache()
out_ds = None

### 示例13-5

In [ ]:
#矢量转栅格
def vector_raster(in_data,out_data,template_data):
    from osgeo import gdal,ogr
    
    #利用ogr返回shapefile数据的图层对象
    driver = ogr.GetDriverByName('ESRI Shapefile')
    vector_ds = driver.Open(in_data,0)
    layer = vector_ds.GetLayer(0)

    #获取模板数据（夜间灯光数据）参数
    template_ds = gdal.Open(template_data)
    template_driver = template_ds.GetDriver()
    rows = template_ds.RasterYSize
    cols = template_ds.RasterXSize
    geotransform = template_ds.GetGeoTransform()
    projection = template_ds.GetProjection()

    dst_driver = template_driver
    dst_ds = dst_driver.Create(out_data,cols,rows,1,gdalconst.GDT_Byte)
    dst_ds.SetGeoTransform(geotransform)
    dst_ds.SetProjection(projection)
    gdal.RasterizeLayer(dst_ds,[1],layer,options=["ATTRIBUTE=OBJECTID"])

#分区统计
def zone_statistics(zone_vector,zone_raster,value_raster):
    value_ds = gdal.Open(value_raster)
    value_array = value_ds.ReadAsArray()
    zone_ds = gdal.Open(zone_raster)
    zone_array = zone_ds.ReadAsArray()
    driver = ogr.GetDriverByName('ESRI Shapefile')
    vector_ds = driver.Open(zone_vector,0)
    layer = vector_ds.GetLayer(0)
    names = []
    GDPs = []
    TNLs = []
    for row in layer:
        oid = row.GetField("OBJECTID")
        name = row.GetField("Name")
        names.append([name])
        GDP = row.GetField("GDP2015")
        GDPs.append([GDP])
        id_array = np.where(zone_array==oid,value_array,0)
        TNL = id_array.sum()/1000  #考虑到TNL值较大，每个值除以1000
        TNLs.append([TNL])
    return (names,TNLs,GDPs)

#回归分析与绘制图表
def regr(names,TNLs,GDPs):
    from sklearn import linear_model
    import matplotlib.pyplot as plt
    from matplotlib.font_manager import FontProperties
    plt.rcParams["font.family"] = "simsun"
    font = FontProperties(fname=r"c:\windows\fonts\simhei.ttf",size=28)
    
    linear_regr = linear_model.LinearRegression()
    linear_regr.fit(TNLs,GDPs)
    coef = linear_regr.coef_
    intercept = linear_regr.intercept_
    R2 = linear_regr.score(TNLs,GDPs)
    GDP_pred = linear_regr.predict(TNLs)
    GDP_bias = list(map(lambda x,y:x-y,list(GDP_pred),GDPs))
    max_bias_i = GDP_bias.index(max(GDP_bias))
    min_bias_i = GDP_bias.index(min(GDP_bias))
    
    plt.scatter(TNLs, GDPs, color='black')
    plt.plot(TNLs, GDP_pred, color='blue', linewidth=3)
    plt.xlim(0,450)
    plt.ylim(0,27000)
    plt.xlabel("TNT($10^3$)")
    plt.ylabel("GDP(亿元)")
    plt.text(100,20000,"$y = %4.2fx - %5.2f$"%(coef,abs(intercept)))
    plt.text(100,18000,"$R^2=%4.2f$"%R2)
    x = TNLs[max_bias_i][0]
    y = GDPs[max_bias_i][0]
    name = f"{names[max_bias_i][0]}"
    plt.text(x,y,name)
    x = TNLs[min_bias_i][0]
    y = GDPs[min_bias_i][0]
    name = f"{names[min_bias_i][0]}"
    plt.text(x,y,name)
    plt.show()

import numpy as np
from osgeo import gdal,ogr   
zone_vector =r'c:\data\VIIRS\csj_aea.shp'
zone_raster = r'c:\data\VIIRS\csj_raster_id.tif'  #中间过程数据
value_raster = r"c:\data\VIIRS\processed_image.tif"
vector_raster(zone_vector,zone_raster,value_raster)
names,TNLs,GDPs = zone_statistics(zone_vector,zone_raster,value_raster)    
regr(names,TNLs,GDPs)